In [17]:
# ============================================
print("📦 Installing dependencies...")
!pip install -q mido tensorflow scikit-learn
print("✅ Dependencies installed\n")

📦 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
✅ Dependencies installed



In [18]:
import os
import zipfile
import numpy as np
import mido
from mido import MidiFile, MidiTrack, Message
from pathlib import Path
from typing import List, Tuple
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import MinMaxScaler
import traceback

In [19]:
# Konfigurasi Global Awal
DATA_PATH = '/content/chillhop'
ZIP_PATH = '/content/chillhopdata.zip'
EXTRACT_PATH = '/content'

In [20]:
# ============================================
# CLASS DEFINITIONS
# ============================================
class MusicDataProcessor:
    """Class untuk memproses data MIDI"""

    def __init__(self, data_path: str, sequence_length: int = 50):
        self.data_path = Path(data_path)
        self.sequence_length = sequence_length
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.notes = []

    def extract_notes_from_midi(self) -> List[int]:
        """Ekstrak notes dari file MIDI"""
        notes = []

        # Pencarian rekursif: mencari semua file MIDI di folder dan subfolder
        midi_files = list(self.data_path.glob("**/*.mid")) + \
                     list(self.data_path.glob("**/*.midi")) + \
                     list(self.data_path.glob("**/*.MID")) + \
                     list(self.data_path.glob("**/*.MIDI"))

        print(f"🎵 Memproses {len(midi_files)} file MIDI dari {self.data_path}...")

        if len(midi_files) == 0:
            raise ValueError(f"Tidak ada file MIDI ditemukan di: {self.data_path}")

        for idx, midi_file in enumerate(midi_files, 1):
            try:
                mid = MidiFile(midi_file)
                file_notes = 0
                for track in mid.tracks:
                    for msg in track:
                        # Hanya ambil not yang mulai dimainkan (note_on dengan velocity > 0)
                        if not msg.is_meta and msg.type == 'note_on' and msg.velocity > 0:
                            notes.append(msg.note)
                            file_notes += 1

                if idx <= 5 or idx % 10 == 0:
                    print(f"   [{idx}/{len(midi_files)}] {midi_file.name}: {file_notes} notes")

            except Exception as e:
                print(f"   ✗ Error processing {midi_file.name}: {e}")

        print(f"\n✅ Total notes extracted: {len(notes):,}")
        if len(notes) == 0:
            raise ValueError("Tidak ada notes yang bisa diekstrak dari semua file yang ditemukan!")

        self.notes = notes
        return notes

    def prepare_sequences(self, notes: List[int] = None) -> Tuple[np.ndarray, np.ndarray]:
        """Membuat sequences untuk training"""
        if notes is None:
            notes = self.notes

        print(f"\n📊 Preparing sequences...")
        print(f"   Sequence length: {self.sequence_length}")

        # Normalisasi
        notes_array = np.array(notes).reshape(-1, 1)
        notes_normalized = self.scaler.fit_transform(notes_array).flatten()

        # Buat sequences
        X, y = [], []
        for i in range(len(notes_normalized) - self.sequence_length):
            X.append(notes_normalized[i:i + self.sequence_length])
            y.append(notes_normalized[i + self.sequence_length])

        X = np.array(X)
        y = np.array(y)
        # Reshape untuk LSTM [samples, timesteps, features]
        X = X.reshape((X.shape[0], X.shape[1], 1))

        print(f"   Total sequences: {len(X):,}")
        print(f"   X shape: {X.shape}")
        print(f"   y shape: {y.shape}")

        return X, y

    def inverse_transform(self, data: np.ndarray) -> np.ndarray:
        """Kembalikan data ke skala asli"""
        return self.scaler.inverse_transform(data.reshape(-1, 1)).flatten()

class MusicLSTMModel:
    """Modern LSTM model untuk music generation"""

    def __init__(self, sequence_length: int = 50):
        self.sequence_length = sequence_length
        self.model = None
        self.history = None

    def build_model(self, lstm_units: List[int] = [256, 256, 128],
                     dropout_rate: float = 0.3,
                    learning_rate: float = 0.001) -> models.Sequential:
        """Build model dengan arsitektur modern"""

        print("\n🏗️ Building model...")

        model = models.Sequential(name="MusicLSTM")

        # First LSTM layer
        model.add(layers.LSTM(
            lstm_units[0],
            input_shape=(self.sequence_length, 1),
            return_sequences=True,
            kernel_regularizer=keras.regularizers.l2(0.001)
        ))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))

        # Middle LSTM layers
        for units in lstm_units[1:-1]:
            model.add(layers.LSTM(
                units,
                return_sequences=True,
                kernel_regularizer=keras.regularizers.l2(0.001)
            ))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(dropout_rate))

        # Last LSTM layer
        model.add(layers.LSTM(
            lstm_units[-1],
            return_sequences=False,
            kernel_regularizer=keras.regularizers.l2(0.001)
        ))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))

        # Dense layers
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(1, activation='linear')) # Output 1 not dinormalisasi

        # Compile
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer,
            loss='huber',
            metrics=['mae']
        )

        self.model = model
        return model

    def train(self, X_train: np.ndarray, y_train: np.ndarray,
              X_val: np.ndarray = None, y_val: np.ndarray = None,
              epochs: int = 50, batch_size: int = 64) -> None:
        """Train model dengan callbacks"""

        print("\n🎯 Training model...")

        callback_list = [
            callbacks.EarlyStopping(
                monitor='val_loss' if X_val is not None else 'loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss' if X_val is not None else 'loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            callbacks.ModelCheckpoint(
                'best_music_model.keras',
                monitor='val_loss' if X_val is not None else 'loss',
                save_best_only=True,
                verbose=0
            )
        ]

        validation_data = (X_val, y_val) if X_val is not None else None

        self.history = self.model.fit(
            X_train, y_train,
            validation_data=validation_data,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callback_list,
            verbose=1
        )

    def generate_music(self, seed_sequence: np.ndarray,
                       num_notes: int = 500,
                      temperature: float = 0.8) -> np.ndarray:
        """Generate musik dengan temperature sampling"""

        print(f"\n🎼 Generating {num_notes} notes...")

        generated = []
        current_sequence = seed_sequence.copy() # Pastikan kita tidak memodifikasi seed asli

        for i in range(num_notes):
            # Predict next note
            prediction = self.model.predict(
                current_sequence.reshape(1, self.sequence_length, 1),
                verbose=0
            )

            # Apply temperature (di sini temperature berfungsi sebagai pembagi untuk scaling output)
            prediction_scaled = prediction[0, 0] / temperature

            generated.append(prediction_scaled)

            # Update sequence: hapus not pertama, tambahkan not prediksi
            current_sequence = np.append(current_sequence[1:], prediction_scaled)

            if (i + 1) % 100 == 0:
                print(f"   Generated: {i + 1}/{num_notes} notes")

        return np.array(generated)

    def save_model(self, filepath: str = 'music_lstm_model.keras'):
        """Save model"""
        self.model.save(filepath)
        print(f"💾 Model saved: {filepath}")

    def load_model(self, filepath: str = 'music_lstm_model.keras'):
        """Load model"""
        self.model = models.load_model(filepath)
        print(f"📂 Model loaded: {filepath}")

class MidiGenerator:
    """Class untuk generate MIDI file"""

    @staticmethod
    def create_midi(notes: List[int],
                    output_path: str = 'generated_music.mid',
                   tempo: int = 500000, # Quarter note = 500ms
                   velocity: int = 80,
                   duration: int = 480) -> None: # Standard MIDI time unit
        """Create MIDI file dari list notes"""

        print(f"\n🎹 Creating MIDI file...")

        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        # Set tempo
        track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))

        for note in notes:
            note = int(np.clip(note, 0, 127)) # Pastikan dalam range 0-127

            # Note on (start playing)
            track.append(Message('note_on', note=note, velocity=velocity, time=0))
            # Note off (stop playing)
            # time=duration berarti not berikutnya dimulai setelah durasi ini
            track.append(Message('note_off', note=note, velocity=velocity, time=duration))

        mid.save(output_path)
        print(f"✅ MIDI saved: {output_path}")
        print(f"   Total notes: {len(notes)}")
        # Estimasi durasi total dalam detik
        print(f"   Duration estimate: ~{len(notes) * duration * (tempo / 1000000) / 480:.1f} seconds")

In [21]:
# ============================================
# MAIN EXECUTION
# ============================================
# Fungsi pembantu untuk mencari folder MIDI yang benar
def find_midi_folder_recursively(start_dir):
    """Mencari subfolder pertama yang berisi file MIDI"""
    for root, dirs, files in os.walk(start_dir):
        midi_count = sum(1 for f in files if f.lower().endswith(('.mid', '.midi')))
        if midi_count > 0:
            return root, midi_count
    return None, 0

def main():
    """Main function"""
    global DATA_PATH
    global ZIP_PATH
    global EXTRACT_PATH

    # Configuration
    SEQUENCE_LENGTH = 50
    EPOCHS = 50
    BATCH_SIZE = 64
    TRAIN_SPLIT = 0.85

    # ============================================
    # VERIFY & EXTRACT FILE
    # ============================================
    print("=" * 60)
    print("🔍 STEP 1: Verifikasi & Extract File (Perbaikan)")
    print("=" * 60)

    # Verifikasi zip file
    if os.path.exists(ZIP_PATH):
        print(f"✅ File zip ditemukan: {ZIP_PATH}")
        file_size = os.path.getsize(ZIP_PATH) / (1024*1024)
        print(f"   Size: {file_size:.2f} MB")

        # Extract jika folder tujuan belum ada
        if not os.path.exists(DATA_PATH):
            print(f"\n📂 Extracting ke {EXTRACT_PATH}...")
            try:
                with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
                    zip_ref.extractall(EXTRACT_PATH)
                print("   ✅ Extract selesai!")
            except Exception as e:
                print(f"   ❌ Gagal extract: {e}")
                exit()
        else:
            print(f"\n✅ Folder {DATA_PATH} sudah ada, skip extraction.")

        # Cek hasil extract dan cari folder dengan MIDI files secara rekursif
        print("\n🔍 Mencari MIDI files di folder hasil extract...")
        correct_path, midi_count = find_midi_folder_recursively(EXTRACT_PATH)

        if correct_path:
            # --- PENTING: Update DATA_PATH dengan path yang benar ---
            DATA_PATH = correct_path
            print(f"✅ Ditemukan {midi_count} MIDI files di DATA_PATH baru: {DATA_PATH}")
        else:
            print("\n❌ ERROR: Tidak ada MIDI files ditemukan di mana pun setelah ekstraksi!")
            print("💡 Cek struktur folder:")
            os.system(f'find {EXTRACT_PATH} -name "*.mid" -o -name "*.midi"')
            raise FileNotFoundError("Tidak ada file MIDI ditemukan di folder hasil ekstraksi.")

    else:
        print(f"❌ File zip TIDAK ditemukan: {ZIP_PATH}")
        print("\n💡 Upload file 'chillhopdata.zip' ke Colab:")
        print("   1. Klik icon folder di sidebar kiri")
        print("   2. Klik icon upload")
        print("   3. Pilih file chillhopdata.zip")
        raise FileNotFoundError("File zip data tidak ditemukan.")

    print(f"\n✅ Data ready di: {DATA_PATH}\n")
    # ============================================

    # 2. Process data
    print("=" * 60)
    print("📊 STEP 2: Processing MIDI Data")
    print("=" * 60)

    processor = MusicDataProcessor(DATA_PATH, SEQUENCE_LENGTH)
    # Akan menggunakan DATA_PATH yang sudah dijamin berisi MIDI files
    notes = processor.extract_notes_from_midi()
    X, y = processor.prepare_sequences(notes)

    # 3. Split data
    print("\n📂 Splitting data...")
    split_idx = int(len(X) * TRAIN_SPLIT)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]

    print(f"   Training set: {X_train.shape[0]:,} sequences")
    print(f"   Validation set: {X_val.shape[0]:,} sequences")

    # 4. Build and train model
    print("\n" + "=" * 60)
    print("🤖 STEP 3: Building & Training Model")
    print("=" * 60)

    model_wrapper = MusicLSTMModel(SEQUENCE_LENGTH)
    model = model_wrapper.build_model(
        lstm_units=[256, 256, 128],
        dropout_rate=0.3,
        learning_rate=0.001
    )

    print("\n📋 Model Summary:")
    model.summary()

    print(f"\n⏱️ Training akan dimulai...")
    print(f"   Epochs: {EPOCHS}")
    print(f"   Batch size: {BATCH_SIZE}")

    model_wrapper.train(
        X_train, y_train,
        X_val, y_val,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )

    # 5. Generate music
    print("\n" + "=" * 60)
    print("🎵 STEP 4: Generating Music")
    print("=" * 60)

    # Random seed dari validation data
    seed_idx = np.random.randint(0, len(X_val))
    seed_sequence = X_val[seed_idx].flatten()

    generated_normalized = model_wrapper.generate_music(
        seed_sequence,
        num_notes=500,
        temperature=0.8
    )

    # Inverse transform
    generated_notes = processor.inverse_transform(generated_normalized)
    # Konversi ke integer
    generated_notes = [int(note) for note in generated_notes]

    # 6. Save as MIDI
    MidiGenerator.create_midi(
        generated_notes,
        output_path='generated_chillhop.mid',
        tempo=500000,
        velocity=80,
        duration=480
    )

    # 7. Save model
    model_wrapper.save_model('final_music_model.keras')

    # 8. Download files (Google Colab)
    print("\n" + "=" * 60)
    print("📥 STEP 5: Download Files")
    print("=" * 60)

    try:
        from google.colab import files
        print("Download hasil ke komputer Anda:")
        print("\n1. Model file:")
        files.download('final_music_model.keras')
        print("\n2. Generated music:")
        files.download('generated_chillhop.mid')
        print("\n✅ Download complete!")
    except ImportError:
        print("⚠️ Bukan Google Colab environment")
        print("Files tersimpan di:")
        print(f"   - final_music_model.keras")
        print(f"   - generated_chillhop.mid")

    print("\n" + "=" * 60)
    print("🎉 SELESAI! Music Generation Complete!")
    print("=" * 60)
    print("\n📝 Summary:")
    print(f"   ✓ Model trained: final_music_model.keras")
    print(f"   ✓ Music generated: generated_chillhop.mid")
    print(f"   ✓ Total notes: {len(generated_notes)}")
    print(f"   ✓ Final DATA_PATH: {DATA_PATH}")

In [22]:
# ============================================
# RUN THE PIPELINE
# ============================================
if __name__ == "__main__":
    print("\n" + "🎵" * 30)
    print("  MUSIC GENERATION WITH LSTM")
    print("🎵" * 30 + "\n")

    try:
        main()
    except Exception as e:
        print(f"\n❌ FATAL ERROR: {e}")
        # Cetak traceback hanya jika itu bukan kesalahan yang diantisipasi (seperti FileNotFoundError)
        if not isinstance(e, (FileNotFoundError, ValueError)):
            traceback.print_exc()

        print("\n💡 Troubleshooting:")
        print("   1. Pastikan file chillhopdata.zip sudah diupload ke /content/")
        print("   2. Cek apakah file zip tersebut memang berisi file MIDI.")
        print("   3. Restart runtime jika ada memory error.")


🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵
  MUSIC GENERATION WITH LSTM
🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵

🔍 STEP 1: Verifikasi & Extract File (Perbaikan)
✅ File zip ditemukan: /content/chillhopdata.zip
   Size: 0.04 MB

📂 Extracting ke /content...
   ✅ Extract selesai!

🔍 Mencari MIDI files di folder hasil extract...
✅ Ditemukan 93 MIDI files di DATA_PATH baru: /content

✅ Data ready di: /content

📊 STEP 2: Processing MIDI Data
🎵 Memproses 93 file MIDI dari /content...
   [1/93] 9.mid: 56 notes
   [2/93] 19.mid: 44 notes
   [3/93] Cymatics - Lofi MIDI 7 - D Maj.mid: 52 notes
   [4/93] Piano MIDI (8).mid: 27 notes
   [5/93] Cymatics - Eternity MIDI 9 - F Min.mid: 49 notes
   [10/93] Cymatics - Lofi MIDI 13 - E Min.mid: 67 notes
   [20/93] Cymatics - Eternity MIDI 22 - B Min.mid: 40 notes
   [30/93] Cymatics - Lofi MIDI 1 - C Maj.mid: 28 notes
   [40/93] Cymatics - Lofi MIDI 11 - E Maj.mid: 24 notes
   [50/93] Rhodes MIDI (9).mid: 12 notes
   [60/93] 14.mid: 21 notes
   [70/93] Piano 2 MIDI.mid: 20 n

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



📋 Model Summary:


Model: "MusicLSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 256)        │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 50, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 50, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

<IPython.core.display.Javascript object>


2. Generated music:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download complete!

🎉 SELESAI! Music Generation Complete!

📝 Summary:
   ✓ Model trained: final_music_model.keras
   ✓ Music generated: generated_chillhop.mid
   ✓ Total notes: 500
   ✓ Final DATA_PATH: /content
